In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
num_of_synthetics = 3
num_of_training = int(0.8*num_of_synthetics)
num_of_val = int(0.2*num_of_synthetics)

In [5]:
# read and write file paths
rpath = "./synthetics"
wpath = './traindata_angle'

# upper and lower limits in time samples for the gather
up = 143
low = 1391
off_limit = 128
for num in range(1, num_of_synthetics+1):

    # load data
    inp_near = np.load(rpath+'/input_near/inp_near_'+str(num)+'.npy')
    targ_near = np.load(rpath+'/target_near/targ_near_'+str(num)+'.npy')
    inp_mid = np.load(rpath+'/input_mid/inp_mid_'+str(num)+'.npy')
    targ_mid = np.load(rpath+'/target_mid/targ_mid_'+str(num)+'.npy')
    inp_far = np.load(rpath+'/input_far/inp_far_'+str(num)+'.npy')
    targ_far = np.load(rpath+'/target_far/targ_far_'+str(num)+'.npy')

    
    # crop data into dimensions appropriate for network
    inp_near = inp_near[up:low,:off_limit]
    targ_near = targ_near[up:low,:off_limit]
    inp_mid = inp_mid[up:low,:off_limit]
    targ_mid = targ_mid[up:low,:off_limit]
    inp_far = inp_far[up:low,:off_limit]
    targ_far = targ_far[up:low,:off_limit]

    
    # make stacks for angular ranges
    stacks = np.zeros((6,low-up))
    data = [inp_near, inp_mid, inp_far, targ_near, targ_mid, targ_far]
    for d,i in zip(data, range(6)):        
    
        stacks[i,:] = np.sum(d, axis=1) / np.count_nonzero(d, axis=1)
        
    stacks = np.nan_to_num(stacks, nan=0.0)
    # normalize
    stacks = (stacks - stacks.mean()) / stacks.std()
    
    # combine into triplet of near-mid-far angle stack traces
    inp = np.stack((stacks[0,:], stacks[1,:], stacks[2,:]), axis=1)
    targ = np.stack((stacks[3,:], stacks[4,:], stacks[5,:]), axis=1)
    
    # save training data
    if num <= num_of_training:
        np.save(wpath+'/before/inp_batch_'+str(num)+'.npy', inp)
        np.save(wpath+'/after/targ_batch_'+str(num)+'.npy', targ)
    
    # save validation data
    else:
        np.save(wpath+'/val/before/inp_batch_'+str(num-num_of_training)+'.npy', inp)
        np.save(wpath+'/val/after/targ_batch_'+str(num-num_of_training)+'.npy', targ)
    
    print(num)

1
2
3
